In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
customer_rep = Path.cwd() / 'data' / 'processed' / 'customer_rep_data.xlsx'
invoice_data = Path.cwd() / 'data' / 'raw' / 'invoice_data.xlsx'
report_file = Path.cwd() / 'reports' / 'commission_report.xlsx'

In [3]:
raw_df = pd.read_excel(customer_rep, usecols='A,B,F,E,H,I,J')

In [ ]:
raw_df.head()

In [ ]:
# Just get the columns we need
customers = raw_df.copy()

In [ ]:
customers.head()

In [ ]:
customers.shape

In [ ]:
invoice = pd.read_excel(invoice_data)

In [ ]:
invoice.head()

In [ ]:
invoice.shape

In [ ]:
invoice['invoice_total'].sum()

In [ ]:
# Merge in Customer name, channel and region
# Different column names for left and right
sales_detail = pd.merge(invoice, customers, left_on='cust_num', right_on='account_num')

In [ ]:
sales_detail.shape

In [ ]:
sales_detail.head()

In [ ]:
sales_detail['invoice_total'].sum()

In [ ]:
commission_rate = 0.0079

In [ ]:
sales_detail['commission'] = commission_rate * sales_detail['invoice_total']

In [ ]:
sales_detail['commission'].sum()

In [ ]:
sales_detail.groupby(['sku']).agg({'commission': 'sum'}).sort_values(by=['commission'], ascending=False)

In [ ]:
sales_detail.groupby(['last_name']).agg({'commission': 'sum'})

In [ ]:
sales_detail.groupby(['last_name', 'channel']).agg({'commission': 'sum'})

In [ ]:
pd.pivot_table(data=sales_detail, index=['last_name'], columns=['channel'], values='commission', aggfunc='sum').round(0)

In [ ]:
# What happens if we adjust the partner & reseller commision rates down?
# Sales team does not do as much work for these channels
partner_rate = 0.001
reseller_rate = 0.0026

In [ ]:
# Add the commission rate as a column with default of the base
sales_detail['commission_rate'] = commission_rate

In [ ]:
partners = (sales_detail['channel'] == 'partner')
resellers = (sales_detail['channel'] == 'reseller')
retail = (sales_detail['channel'] == 'retail')

In [ ]:
partners

In [ ]:
sales_detail.loc[partners, 'commission_rate'] = partner_rate
sales_detail.loc[resellers, 'commission_rate'] = reseller_rate

In [ ]:
sales_detail.head()

In [ ]:
sales_detail['commission_rate'].value_counts()

In [ ]:
# Unlike excel - you need to tell it to calculate
sales_detail['commission'] = sales_detail['commission_rate'] * sales_detail['invoice_total']

In [ ]:
sales_detail.head()

In [ ]:
sales_detail['commission'].sum()

In [ ]:
pd.pivot_table(data=sales_detail,
               index=['last_name'],
               columns=['channel'],
               values='commission',
               margins=True,
               aggfunc='sum').round(0)

In [ ]:
# Shifted things around. Maybe bump up to a different rate?
# Try a loop
retail_rates = [0.01, 0.012, 0.014, 0.016, 0.017]
for rate in retail_rates:
    sales_detail.loc[retail, 'commission_rate'] = rate
    sales_detail['commission'] = sales_detail['commission_rate'] * sales_detail['invoice_total']
    print(f"Total Commmissions {sales_detail['commission'].sum():,.0f} retail rate= {rate}")
    

In [ ]:
pd.pivot_table(data=sales_detail,
               index=['last_name'],
               columns=['channel'],
               values='commission',
               margins=True,
               aggfunc='sum').round(0)

In [ ]:
sales_detail.groupby(['last_name', 'state']).agg({'commission': 'sum'})

In [ ]:
# Move VA and WV to the NE region
# Move Nebraska to the W

In [ ]:
VA_WV = sales_detail['state'].isin(['VA', 'WV'])
NEB = sales_detail['state'].isin(['NE'])
sales_detail.loc[VA_WV, ['Region', 'last_name', 'first_name']] = ('NORTHEAST', 'Muniz', 'Shannon')
sales_detail.loc[NEB, ['Region', 'last_name', 'first_name']] = ('WEST', 'Malcolm', 'Leonard')

In [ ]:
commission_by_channel = pd.pivot_table(data=sales_detail,
                                       index=['last_name'],
                                       columns=['channel'],
                                       values='commission',
                                       margins=True,
                                       fill_value=0,
                                       aggfunc='sum').round(0)

In [ ]:
commission_by_channel

In [ ]:
commission_rates = sales_detail.groupby(['channel']).agg({'commission_rate': 'mean'})
commission_rates

In [ ]:
new_alignments = sales_detail.groupby(['last_name', 'state']).agg({'account_num': 'size', 'invoice_total': 'sum'})
new_alignments

In [ ]:
# Write out the new rates, expected commissions and alignment

In [ ]:
output = pd.ExcelWriter(report_file)

In [ ]:
# Format the numbers
commission_by_channel.to_excel(output)
commission_rates.to_excel(output, startcol=6)
new_alignments.to_excel(output, startcol=9)

In [ ]:
output.save()